In [1]:
from sklearn.externals import joblib

In [2]:
clf = joblib.load('classify.model')
cate_dict = joblib.load('cate_dict.dat')
vectorizer = joblib.load('vectorizer.dat')

In [3]:
joblib.dump(clf,'n_classify.model')
joblib.dump(cate_dict,'n_cate_dict.dat')
joblib.dump(vectorizer,'n_vectorizer.dat')

['n_vectorizer.dat']

In [4]:
cate_id_name_dict = dict(map(lambda (k,v):(v,k),cate_dict.items()))

In [5]:
from konlpy.tag import Twitter; t = Twitter()
from nltk import pos_tag, word_tokenize
from nltk.util import ngrams

In [6]:
brackets = ["[", "{", "("]
brackete = ["]", "}", ")"]
def getBrac(name, chk=True):
    brac = {True:"", False:""}
    for c in name:
        chk = not c in brackets and chk
        if c not in brackets and c not in brackete: brac[chk] += c
        else: brac[chk] += " "; brac[not chk] += " "
        chk = c in brackete or chk
    return (brac[True], brac[False])

In [7]:
gram_cnt = 2
chk_NC = ["NN", "CD"]
chk_NA = ["Noun"]
chk_NAN = ["Noun", "Number"]

getNLTK = lambda name, chk : [token[0] for token in pos_tag(name) if token[1][:2] in chk]
getKONL = lambda name, chk : [token[0] for token in t.pos(name) if token[1] in chk]
getTokn = lambda name, chk_ko, chk_en : [token[0] for token in t.pos(name) if (token[1] == "Alpha") or token[1] in chk_ko]
getGram = lambda name, cnt : [" ".join(["".join(grams)]) for grams in ngrams(name, cnt)]
def getName(name):
	brac = getBrac(name)
	noun = getTokn(brac[0], chk_NAN, chk_NC) + getTokn(brac[1], chk_NAN, chk_NC)
	gram = getGram(noun, min(gram_cnt, max(len(noun)-1, 1)))
	return " ".join(list(set(noun)) + list(set(gram)))

In [8]:
getImageName = lambda name : name.split('/')[-1][:-4]

In [9]:
from classify_image_module import getImageJson
def getImge(name):
    return getImageJson(name)

In [10]:
test_name = u"인텔 인텔 코어i7-4세대 4770K (하스웰) (정품)"
test_imge = u"90985"
name = getName(test_name)
imge = getImge(test_imge)
print "name: ", name
print "image: ", imge

name:  4770 i 코어 4 7 하스웰 K 세대 인텔 정품 i7 4세대 인텔코어 코어i 74 인텔인텔 하스웰정품 4770K K하스웰 세대4770
image:   


In [11]:
pred = clf.predict(vectorizer.transform([name + imge]))[0]
print cate_id_name_dict[pred]

디지털/가전;PC부품;CPU


In [12]:
name_test = u"(퀸)국내생산100% 바자르 번아웃극세사 차렵세트"
imge_test = "http://image.hnsmall.com/images/goods/477/11924477_g.jpg"
name = getName(name_test)
imge = getImge(getImageName(imge_test))
print "name: ", name
print "image: ", imge

name:  생산 번 아웃 차렵 국내 세트 바자르 극세사 퀸 100 생산100 아웃극세사 차렵세트 세트퀸 번아웃 100바자르 국내생산 바자르번 극세사차렵
image:   quilt


In [13]:
pred = clf.predict(vectorizer.transform([name + imge]))[0]
print cate_id_name_dict[pred]

패션의류;남성언더웨어/잠옷;잠옷/홈웨어


In [ ]:
from IPython.display import clear_output

In [ ]:
from bottle import route, run, template,request,get, post

import  time
from threading import  Condition
_CONDITION = Condition()

count = 0

@route('/classify')
def classify():
    global count
    count += 1
    print count
    if not count % 100:
        clear_output()
    img = request.GET.get('img','')
    name = request.GET.get('name', '')
    # getImge(getImageName(img))
    pred = clf.predict(vectorizer.transform([getName(name.decode('utf-8'))]))[0]
    return {'cate':cate_id_name_dict[pred]}

run(host='0.0.0.0', port=8887)


10.0.2.2 - - [22/Aug/2016 10:39:32] "GET /classify?name=%ED%8E%9C%EC%8B%B1[5%EB%B6%80%EC%A7%84]%EC%BB%B7%ED%8C%85%205%EB%B6%80p(ver.dark%20jean)%ED%8E%98%EC%9D%B8%ED%8C%85%20%EC%9B%8C%EC%8B%B1%EC%9D%98%ED%9D%AC%EC%86%8C%EA%B0%80%EC%B9%98%EC%99%80%20%EB%B9%88%ED%8B%B0%EC%A7%80%ED%95%A8%EC%97%90%EC%8A%A4%ED%8C%90%EB%9F%89%EC%9D%B4%20%EC%A2%8B%EC%95%84%EC%9A%9426~30%EA%B9%8C%EC%A7%80%20%EC%B0%A9%EC%9A%A9%EA%B0%80%EB%8A%A5&img=http://shopping.phinf.naver.net/main_7783737/7783737592.jpg HTTP/1.1" 200 74
10.0.2.2 - - [22/Aug/2016 10:39:32] "GET /classify?name=no.5305%20%EB%84%A4%EC%98%A4%ED%94%84%EB%9E%9C%20%EC%8B%AC%ED%94%8C%20%EC%95%84%EB%B0%A9%EC%BD%94%ED%8A%B8[2color%20/%20%EA%B0%84%EC%A0%88%EA%B8%B0~%EB%B4%84%20%EA%B0%80%EC%9D%84%EA%B9%8C%EC%A7%80%20%EC%B0%A9%EC%9A%A9%EA%B0%80%EB%8A%A5%ED%95%A9%EB%8B%88%EB%8B%A4]&img=http://shopping.phinf.naver.net/main_8238777/8238777002.jpg HTTP/1.1" 200 74
10.0.2.2 - - [22/Aug/2016 10:39:32] "GET /classify?name=%EC%9D%B4%EC%8A%A4%EB%85%B8%20%EB%84%A4

3001
3002
3003
3004
3005
3006
3007
3008
3009


10.0.2.2 - - [22/Aug/2016 10:39:32] "GET /classify?name=%EB%A0%88%EB%85%B8%EB%A7%88(%EB%84%A5%ED%83%80%EC%9D%B4)%20[%EB%A1%AF%EB%8D%B0%EB%B0%B1%ED%99%94%EC%A0%90]%EC%A0%84%EC%B2%B4%EC%A0%81%EC%9C%BC%EB%A1%9C%20%EC%82%AC%EB%9D%BC%EC%82%AC%20%EB%AC%B8%EC%96%91%EC%97%90%20%EC%84%A0%EB%AA%85%ED%95%9C%20%EC%B9%BC%EB%9D%BC%EA%B0%90%EC%9D%B4%20%EB%8F%8B%EB%B3%B4%EC%9D%B4%EB%8A%94%20%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%83%88%20%EB%84%A5%ED%83%80%EC%9D%B4&img=http://shopping.phinf.naver.net/main_7965752/7965752864.jpg HTTP/1.1" 200 123
10.0.2.2 - - [22/Aug/2016 10:39:32] "GET /classify?name=%EA%B0%80%ED%8A%B8%20%ED%8C%A8%EB%94%A9%20%EC%BD%94%ED%8A%B8[2color]%ED%81%B4%EB%9E%98%EC%8B%9D%ED%95%9C%20%EB%B6%84%EC%9C%84%EA%B8%B0%EC%99%80%20%EB%94%B0%EC%8A%A4%ED%95%9C%20%EB%B3%B4%EC%98%A8%EC%84%B1%20GOOD!%EA%B8%B0%EC%A1%B4%EA%B0%80%2060,000%EC%9B%90&img=http://shopping.phinf.naver.net/main_8232395/8232395461.jpg HTTP/1.1" 200 74
10.0.2.2 - - [22/Aug/2016 10:39:32] "GET /classify?name=[CROOKLYN]%20[%ED%81%AC

3010
3011
3012
3013


10.0.2.2 - - [22/Aug/2016 10:39:32] "GET /classify?name=%EC%9D%B8%EB%94%94%EA%B3%A0%20%EB%8D%B0%EB%8B%98%20%EB%B8%94%EB%A0%88%EC%9D%B4%EC%A0%B8%EC%8A%A4%ED%8A%B8%EB%9D%BC%EC%9D%B4%ED%94%84%20%ED%8C%A8%ED%84%B4!!%EC%9B%8C%EC%8B%B1%EB%8D%B0%EB%8B%98%EA%B0%80%EA%B3%B52color%EB%81%8A%EC%9E%84%EC%97%86%EC%9D%B4%20%EC%82%AC%EB%9E%91%EB%B0%9B%EA%B3%A0%20%EC%9E%88%EB%8A%94%20%EB%8D%B0%EB%8B%98%EC%86%8C%EC%9E%AC%EC%9D%98%20%EC%9E%90%EC%BC%93!!%ED%95%8F%EC%9D%84%20%EC%9E%A1%EC%95%84%EC%A3%BC%EC%96%B4%20%EB%8B%A8%EC%A0%95%ED%95%9C%20%ED%95%8F%20%EB%9D%BC%EC%9D%B8%EA%B3%BC%20%EA%B9%94%EB%81%94%ED%95%9C%20%EC%9D%B4%EB%AF%B8%EC%A7%80%20%EC%97%B0%EC%B6%9C%EC%9D%B4%20%EA%B0%80%EB%8A%A5%ED%95%98%EC%97%AC%20%ED%99%9C%EC%9A%A9%EB%8F%84%20%EB%86%92%EC%9D%80%20%EC%95%84%EC%9D%B4&img=http://shopping.phinf.naver.net/main_6997619/6997619229.jpg HTTP/1.1" 200 74
10.0.2.2 - - [22/Aug/2016 10:39:32] "GET /classify?name=%ED%85%8C%EC%9D%B4%EB%A8%BC%20%ED%97%A8%EB%A6%AC%EB%84%A5%20%EA%B8%B4%ED%8C%94%ED%8B%B0[2color

...

3022
3023
3024
3025


10.0.2.2 - - [22/Aug/2016 10:39:33] "GET /classify?name=RED-7%20%EC%8A%A4%ED%8C%90%20%ED%95%98%ED%94%84%20%EB%8D%B0%EB%8B%98%ED%8C%AC%EC%B8%A0%EC%9C%A0%ED%96%89%EC%9D%84%20%ED%83%80%EC%A7%80%20%EC%95%8A%EB%8A%94%20%ED%8C%A8%ED%84%B4%EC%9C%BC%EB%A1%9C%20%EC%96%B8%EC%A0%9C%EB%82%98%20%ED%8E%B8%ED%95%98%EA%B2%8C%20%EC%A6%90%EA%B2%A8%EC%9E%85%EA%B8%B0%20%EC%A2%8B%EC%9D%80%20%ED%8C%AC%EC%B8%A0(S/M/L)&img=http://shopping.phinf.naver.net/main_6978300/6978300964.jpg HTTP/1.1" 200 74
10.0.2.2 - - [22/Aug/2016 10:39:33] "GET /classify?name=%EB%9D%BC%EC%9D%B8%EC%9D%B4%20%EB%8F%8B%EB%B3%B4%EC%9D%B4%EB%8A%94!%20%EB%B3%BC%EB%A5%A8%20%EC%8A%AC%EB%A6%BC%ED%95%8F%EC%88%84%EC%B9%B4%EB%9D%BC%20%EC%BB%B7%ED%8C%85%EB%9F%AC%ED%94%8C%20%EC%A7%9A%EC%97%85%20%EB%8B%88%ED%8A%B8%EC%9E%90%EC%BC%93/JK537&img=http://shopping.phinf.naver.net/main_8303883/8303883844.jpg HTTP/1.1" 200 74


3026
3027
3028
3029
3030
3031
3032


10.0.2.2 - - [22/Aug/2016 10:39:33] "GET /classify?name=%EC%B2%B4%ED%81%AC%EC%9B%90%ED%94%BC%EC%8A%A4(1color)%EB%A7%A8%ED%88%AC%EB%A7%A8,%EB%8B%88%ED%8A%B8%20%EB%AA%A8%EB%91%90%20%EB%A0%88%EC%9D%B4%EC%96%B4%EB%93%9C%EA%B0%80%EB%8A%A5%ED%95%9C%20%EC%B2%B4%ED%81%AC%EC%9B%90%ED%94%BC%EC%8A%A4:)&img=http://shopping.phinf.naver.net/main_8022648/8022648255.jpg HTTP/1.1" 200 80
10.0.2.2 - - [22/Aug/2016 10:39:33] "GET /classify?name=[Vane-OP556]%20Black%20line%20dress-%EC%9A%B0%EC%95%84%ED%95%9C%20%EB%A9%8B!%20%EC%84%B8%EB%A0%A8%EB%90%9C%20%EB%9D%BC%EC%9D%B8%EC%9D%B4%20%EB%8F%8B%EB%B3%B4%EC%97%AC%EC%9A%94!&img=http://shopping.phinf.naver.net/main_7884749/7884749622.jpg HTTP/1.1" 200 80
10.0.2.2 - - [22/Aug/2016 10:39:33] "GET /classify?name=%EB%8D%B0%EB%8B%88%EC%8A%A4[%EC%85%94%EC%B8%A0%EC%9E%90%EC%BC%93]%EC%A7%80%ED%8D%BC%20%EC%96%B8%EB%B0%9Cjp%EC%BD%94%ED%8A%BC%EC%95%88%EA%B0%90%EC%9D%B4%20%ED%95%9C%EA%B2%B9%EB%8D%94%20%EB%8D%A7%EB%8C%80%EC%96%B4%EC%A0%B8%EB%82%A8%EB%B0%A9%EB%B3%B4%EB%8B%A4

3033
3034
3035
3036
3037
